# Simple Chain  

In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt = PromptTemplate(
    template='Generate 5 interesting facts about {topic}',
    input_variables=['topic']
)

model = ChatGroq(model="gemma2-9b-it")

parser = StrOutputParser()

chain = prompt | model | parser

chain.invoke({"topic": "apple"})


'Here are 5 interesting facts about apples:\n\n1. **Apples are technically berries!**  This is because they develop from a single ovary of a flower and contain seeds.  \n2. **Ancient Romans used apples for currency.**  They were so valuable that they even had a saying, "An apple a day keeps the doctor away," which dates back to Roman times.\n3. **The world\'s largest apple tree is in the U.S.** Located in  Bartlett, Tennessee, this apple tree is estimated to be over 100 years old and covers an acre of land. \n4. **There are over 7,500 varieties of apples!**  While only a few are widely popular, there\'s a huge diversity of apples out there, each with unique flavors and textures. \n5. **Apples float!**  Due to their air pockets, apples are less dense than water and will float.\n\n\n\nLet me know if you\'d like to know more about apples! 🍎 \n'

### Visualize the chain

In [3]:
#pip install grandalf

Note: you may need to restart the kernel to use updated packages.


In [4]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


# Sequential Chain

Topic → Detailed Report → LLM → Prompt 2 → 5 point summary

In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
load_dotenv()

prompt1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a 5 pointer summary from the following text \n {text}',
    input_variables=['text']
)

In [6]:
model = ChatGroq(model="gemma2-9b-it")

parser = StrOutputParser()

In [7]:
chain = prompt1 | model | parser | prompt2 | model | parser

chain.invoke({"topic": "apple"})

"Here's a 5-point summary of the Apple Inc. report:\n\n1. **Evolution of a Giant:** Apple's journey began with personal computers, evolved through the iPod and iPhone revolutions, and now encompasses a diverse ecosystem of hardware and software.\n2. **Powerful Business Model:** Apple's success stems from selling premium hardware, a thriving software and services ecosystem (App Store, iCloud, etc.), and strong brand loyalty that drives repeat purchases.\n3. **Exceptional Financial Performance:** Apple consistently reports high revenue and profits, maintains significant cash reserves, and boasts a highly valuable stock.\n4. **Navigating a Competitive Landscape:** While facing rivals like Samsung, Google, and Microsoft, Apple differentiates itself through its brand, design, ecosystem integration, and customer relationships.\n5. **Future Focus and Challenges:** Apple is poised for growth through continued innovation in areas like AI and AR, expanding its services, and targeting emerging ma

# Parallel Chain

In [9]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
load_dotenv()

model1 = ChatGroq(model="gemma2-9b-it")
model2= ChatGroq(model="llama-3.1-8b-instant")

prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

In [21]:
prompt3= PromptTemplate(
    template="Merge the provided notes and quiz into a single document \n notes: {notes} and {quiz}",
    input_variables=["notes", "quiz"]
)

In [15]:
parser = StrOutputParser()

### Make a parallel chain:

Make 2 chains & pass them in RunnableParallel & name them

In [30]:
from langchain_core.runnables import RunnableParallel

notes = prompt1 | model | parser

quiz = prompt2 | model | parser

parallel_chain = RunnableParallel({
    "notes": notes,
    "quiz": quiz
})

In [31]:
merge_chain = prompt3 | model2 | parser

chain = parallel_chain | merge_chain

Invoke the chain

In [32]:
text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

result =chain.invoke({"text": text})

In [33]:
print(result)

## Support Vector Machines (SVMs) - Notes and Quiz

### What are SVMs?

* Supervised learning methods for:
    * Classification
    * Regression
    * Outlier detection

### Advantages:

* Effective in high dimensional spaces.
* Work well even when dimensions > samples.
* Memory efficient (uses a subset of training data - support vectors).
* Versatile: different kernel functions can be used.

### Disadvantages:

* **Overfitting risk:** When features >> samples, careful kernel selection and regularization are crucial.
* **No direct probability estimates:** Calculated using expensive cross-validation.

### Input Data:

* Can handle dense (numpy arrays) or sparse (scipy sparse) vectors.
* **Important:** Must train on the same data type you want to predict with.
* **Best performance:** C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.

### Quiz

**1. What are Support Vector Machines (SVMs) used for?**
**Answer:** SVMs are used for classification, regre

In [25]:
chain.get_graph().print_ascii()

          +---------------------------+            
          | Parallel<notes,quiz>Input |            
          +---------------------------+            
                ***             ***                
              **                   **              
            **                       **            
+----------------+              +----------------+ 
| PromptTemplate |              | PromptTemplate | 
+----------------+              +----------------+ 
          *                             *          
          *                             *          
          *                             *          
    +----------+                  +----------+     
    | ChatGroq |                  | ChatGroq |     
    +----------+                  +----------+     
          *                             *          
          *                             *          
          *                             *          
+-----------------+            +-----------------+ 
| StrOutputP

# Conditional Chain

In [34]:
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

True

In [68]:
model = ChatGroq(model="gemma2-9b-it")

parser = StrOutputParser()

In [40]:
prompt1 = PromptTemplate(
    template= "Classify the feedback into positive or negative. \n feedback : {feedback}",
    input_variables= ["feedback"]
)

classifier_chain = prompt1 | model | parser

In [46]:
classifier_chain.invoke({"feedback": "this is terrible smartphone"})

'The feedback "This is a terrible smartphone" is **negative**. \n'

### Convert this to structured output

In [69]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

class Feedback(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field (description="Give the sentiment of the feedback")

parser2 = PydanticOutputParser(pydantic_object=Feedback)    

In [70]:
prompt1 = PromptTemplate(
    template= "Classify the feedback into positive or negative. \n feedback : {feedback} \n {format_instruction}",
    input_variables= ["feedback"],
    partial_variables= {"format_instruction": parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

print(classifier_chain.invoke({"feedback": "this is terrible smartphone"}))

sentiment='negative'


### Branching:

In [71]:
pos_fb = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

neg_fb = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

In [81]:
branch_chain = RunnableBranch(
    (lambda x: x.sentiment == 'positive', pos_fb | model2 | parser),
    (lambda x: x.sentiment == 'negative', neg_fb | model2 | parser),
    RunnableLambda (lambda x: "could not find sentiment")  # default branch if none match
)

In [82]:
chain = classifier_chain | branch_chain

In [83]:
result = chain.invoke({'feedback': 'This is a terrible phone'})
print(result)

"I'm sorry to hear that our product/service didn't meet your expectations. Can you please provide more details about your experience so we can better understand what went wrong? This will help us to improve our product/service and prevent similar issues in the future."


In [84]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
